## Metagenomics 101 course
- https://staff.fnwi.uva.nl/a.u.s.heintzbuschart/metagenomics_00.html  

## Lecture 1  

- lecture 1: https://video.uva.nl/media/Metagenomics+101+-+1%7C+overview+-+private+version/0_pvgszh0o 
    - definition of metagenomics
    - brief history of metagenomics
    - glimpse into raw data
    - typical steps in an analysis
- Tutorial 1:
    - Set up: didn't initialize miniconda since it had already been used before and saved data in personal directory.
        - First test-run of IMP3 in Crunchomics - dry run to test configuration_
            - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -d my.config.yaml
            - desired output: This was a dry-run (flag -n). The order of jobs does __not__ reflect the order of execution.
        - Submitted the first test run to the compute nodes of crunchomics server in node 3 with omics-cn003.
            - first check the available nodes: sinfo -o "%n %e %m %a %c %C" 
        - IMP3 inputs: data and configuration file 
            - my.config.yaml configuration file is the only required argument and specifies the input data. 
            - code used to commit the real run to the compute node, which includes more arguments than the previous dry run: 
                - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -c -r -n TESTRUN -b omics-cn002 my.config.yaml

## Lecture 2

- Lecture 2: Raw Data Preprocessing and QC 
    - .fastq files tools resource: http://barcwiki.wi.mit.edu/wiki/SOPs/qc_shortReads 
    - brief look into different sequencing techniques
    - what can go wrong when sequencing
    - sequencing quality scores
    - quality reports
    - adapter, spike-in, and host genome removal (the host microbiome needs to be removed because you are interested in the other genomes in the sample)
- Tutorial 2: Run Preprocessing on Toy Data
    - Finding Novaseq file in reads data - the GCEF.test.r1.fq and GCEF.test.r2.fq are novaseq
    - code used to modify the configuration file __prep.1.config.yaml__ to use datasets COSMIC_vag.test.r1.fq and COSMIC_vag.test.r2.fq instead of the default test data, by manually changing inside the configuration file by using nano command: 
        - raws:
            Metagenomics: "/zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/COSMIC_vag.test.r1.fq /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/COSMIC_vag.test.r2.fq"
    - made a copy of prep.1.config.yaml to be able to run both configuration at the same time and have their own output directories. 
    - code used to modify the configuration file __prep.2.config.yaml__ to use datasets GCEF.test.r1.fq and GCEF.test.r2.fq instead of the default test data, by manually changing inside the configuration file by using __nano__ command: 
        - raws:
            Metagenomics: "/zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/GCEF.test.r1.fq /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/GCEF.test.r2.fq"
        - also changed the output directory to __IMP3_test_preprocessing_novaseq__, and nextseq argument to __true__. 
    - performed the dry run and obtained desired output:
        - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -d prep.1.config.yaml 
        - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -d prep.2.config.yaml 
    - both runs were commited to node 2:
            - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -c -r -n TESTRUN -b omics-cn002 prep.1.config.yaml
            - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -c -r -n TESTRUN -b omics-cn002 prep.2.config.yaml
    - The dataset COSMIC_vag.test.r1|2.fq comes from a sample that contains still host genomics DNA
        - prep.1.config.yaml was copied to prep.3.config.yaml and a new configuration was specified with the same test data, a new output directory __IMP3_test_preprocessing_hg38filtering__, and the filtering was specified to hg38 to filter out the human genome 
        - dry run was performed and successful 
        - run was submitted to node 4: 
            - /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -c -r -n TESTRUN -b omics-cn004 prep.3.config.yaml

## Lecture 3

- Lecture 3: Mapping Metagenomics read data to reference genomes 
    - why mapping
    - bioinformatics for sequence alignment
    - look at output (alignments) created by read mappers
    - what can be done with the output  
- Tutorial 3:
    - not IMP3 pipeline but the commands on their own  
    - environments IMP3 provides on crunchomics to preprocess data with BWA-MEM - Burrows Wheeler algoritm (transform) - Maximum Exact Matches 
    1. Running BWA-MEM indexing 
        - build index of references (don't do on the frontend if using a real genomes, okay now bc test data)
        - cp /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/REFERENCES/mg.assembly.merged.fa . (copied to my personal directory)
        - conda activate /zfs/omics/projects/metatools/TOOLS/IMP3/conda/76e02d85877600c41ac84cb7bc27a189
        - bwa index mg.assembly.merged.fa 
    2. Running mapping with BWA-MEM 
        - bwa mem -v 1 -t 1  mg.assembly.merged.fa /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/test.mg.1.fastq /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/test.mg.2.fastq >> test_alignment.sam
        - you can run other data, for examble: COMSIC_vag.test.r1|r2.fq with the hg38.fa - first need to find hg38.fa file to copy to personal directory and index it 
    3. Take a look at the alignment file 
        - output of BWA-MEM is an alignment file in .sam format. Alignment information is at the end of the file 
        - tail test_alignment.sam 
    4. Converting the alignment file to a binary (compressed) format 
        - convert to .bam format because the .sam is very big and you don't want to store on own disk 
        - samtools view --threads 1 -bS test_alignment.sam > test_alignment.bam
        - can also be donde in one step with the mapping:                 - bwa mem -v 1 -t 1  /zfs/omics/projects/metatools/DB/filtering/hg38.fa /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/COSMIC_vag.test.r1.fq /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/COSMIC_vag.test.r2.fq |\
                    samtools view --threads 1 -bS - > COSMIC_alignment.bam
        - can look at a .bam file with samtools view: samtools view test_alignment.bam | less
    5. Filtering the .bam file 
        - also using samtools view, use -f 4 to find only reads which don't map to reference 
        - samtools view -f 4 test_alignment.bam | less 
        - this shows many of the values to be 0 
        - -F 4 give reads which map (careful, there's also a second read)
        - more samtools flags: https://broadinstitute.github.io/picard/explain-flags.html
        - samtools manual: http://www.htslib.org/doc/samtools-view.html 
    6. More complex filtering 
        - used in IMP3 to remove reads mapping to a host genome 
        - samtools merge --threads 1 -u - \
            <(samtools view --threads 1 -u  -f 4 -F 264 COSMIC_alignment.bam) \
            <(samtools view --threads 1 -u -f 8 -F 260 COSMIC_alignment.bam) \
            <(samtools view --threads 1 -u -f 12 -F 256 COSMIC_alignment.bam)  | \
            samtools view --threads 1 -bF 0x800 - | \
            samtools sort --threads 1 -m 3G -n - | \
            bamToFastq -i stdin -fq COSMIC_filtered.r1.fq -fq2 COSMIC_filtered.r2.fq

            conda deactivate 
            (all COSMIC_alignment.bam files are the test_alignment.bam files when doing the alignment of COSMIC files to the hg38.fa reference genome - didn't actually perform that alignment) (figure out what the code on top is doing by looking at the resources of flags and samtools documentation)

## Lecture 4

- Lecture 4: Taxonomy
    - different approaches toprofile microbiomes in terms of the present taxa, using metagenomics read data 
    - introduction into the aims and particular environments 
    - a touch of taxonomy 
    - brief reminder of mapping of metagenomis reads 
    - look at non-alignment based profilers 
    - how taxonomic profilers are benchmarked 
    - what we can do with this output 
- Tutorial 4: Run mOTUs2, MetaPhlan2, and kraken2
    - Use the taxonomic profiles in the IMP3 pipeline as well as MetaPhlan3 (IMP3 in crunchomics and metaphlan3 conda environment)
    - use cd /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS files or the preprocessed files from the last sessions
    - Running taxonomic profiling within __IMP3__ 
        - start IMP3 with an appropriate configuration file since data is preprocessed. Set config file to not do preprocessing, and set input as already preprocessed files
        > cp /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/04_taxonomy1/tax.config.yaml my.tax.config.yaml 
        - change (using nano) to input own data. Can also change the kraken databse to one of the names in /zfs/omics/projects/metatools/DB with the word kraken in it (UHGP_kraken2 for human gut genomes, minikraken2 for an even smaller database; kraken_pfp8 is the current kraken db in configuration). Output directory: test_tax_output
            >raws:
            >>Metagenomics: /home/14108550/personal/IMP3_test_preprocessing/Preprocessing/mg.r1.trimmed.phiX174_filtered.fq
            >>/home/14108550/personal/IMP3_test_preprocessing/Preprocessing/mg.r2.trimmed.phiX174_filtered.fq
            >>/home/14108550/personal/IMP3_test_preprocessing/Preprocessing/mg.se.trimmed.phiX174_filtered.fq
        - first did dry run, which was successful 
            > /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -d my.tax.config.yaml
        - then commited run to node 3 
            > /zfs/omics/projects/metatools/TOOLS/IMP3/runIMP3 -c -r -n TESTTAX -b omics-cn003 my.tax.config.yaml
        - inside the output directory will be another directory Analysis/taxonomy which holds the outputs from mOTUs2 and kraken2 
        - Taxonomic profiling using __MetaPhlan3__ 
            - works very well on human samples and has a strain-level module. 
            - activate the conda environment: 
            > conda activate /zfs/omics/projects/metatools/TOOLS/miniconda3/envs/metaphlan-3.0
            - to get help and documentation on the profiler:
            > metaphlan -h 
            - example to run on filtered test reads:
            > metaphlan /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/FILTERED_READS/mg.r1.trimmed.phiX174_filtered.fq,/zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/FILTERED_READS/mg.r2.trimmed.phiX174_filtered.fq,/zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/FILTERED_READS/mg.se.trimmed.phiX174_filtered.fq --bowtie2out filtered_test.bowtie2.bz2 --input_type fastq -o profile.filtered_test.txt  --bowtie2db /zfs/omics/projects/metatools/DB/metaphlan --unknown_estimation
            - can also run on own filtered reads instead. to run on real data set, put the commands to activate (and deactivate) conda and the metaphlan command into a script that you submit to the cluster. You can then also choose to set the --nproc argument (--nproc 8 to parallelize the profiling) 
            - code for runnin the non-filtered reads 
            > metaphlan /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/test.mg.r1.fastq,/zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/READS/test.mg.r2.fastq --bowtie2out unfiltered_test.bowtie2.bz2 --input_type fastq -o profile.unfiltered_test.txt  --bowtie2db /zfs/omics/projects/metatools/DB/metaphlan --unknown_estimation
            - To merge the profiles from several samples (or in this case, the same sample, but represented by filtered or unfiltered reads), you can use metaphlan’s merge script
            > merge_metaphlan_tables.py profile*filtered_test.txt > merged_abundance.test.tsv
            - to compare the results:
            > less merged_abundance.test.tsv
            > conda deactivate

## Lecture 5

- Metagenomics 101 Lecture 5: Assembly
    - lecture video: https://video.uva.nl/media/Metagenomics+101+-+5%7C+Assembly+-+private+version/0_p8gnde38 
    - why assemble
    - alternatives to assembly
    - how does assembly work
    - how to inspect assemblies
    - what to assemble 
    - stopped at min 17 
- Tutorial 5: Run assemblies in IMP3 
    - assemble with IMP3 and look at diagnostic plots 

## Lecture 6

- Metagenomics Lecture 6: Metagenome Annotation (of assembled contigs)
    - finding bacterial genes
        - protein coding genes
        - rRNAs
    - annotating genes with functions
        - why not just align?
        - HMMs and HMMER
- Tutorial 6: 

## Lecture 7

- Metagenomics 101 Lecture 7 
  - approaches to make (semi-)quantitative profiles of the molecular functions found in metagenomes and how they can be compared across samples
  - review on mapping and annotation
  - gene abundance measures - functional profile calculation:
    - reads per gene/reads per function
    - reads per kilobase
    - copies per million
    - average depth of coverage

- Tutorial: take a look at annotations:
  - bash tricks to get a quick look at annotations: 
    - head annotation_CDS_RNA_hmms.gff
    - tail -n 5 annotation_CDS_RNA_hmms.gff
    - wc -l annotation_CDS_RNA_hmms.gff --> gives a count of annotated sequences 
      - wc stands for word count 
      - -l option prints the number of lines present in a file 
    -  grep "CDS" -c annotation_CDS_RNA_hmms.gff --> 8382 is the number of lines that are CDS type 
      - grep is a command-line utility for searching plain-text data sets for lines that match a regular expression
      - -c flag is used to ask for the total number of lines that contain that string - counts 
    - cut -f 3 annotation_CDS_RNA_hmms.gff | sort | uniq -c
      - what field are in the column 3? 
      - 8382 CDS
      - 1 repeat_region
      - 17 rRNA
      - 1 tmRNA
      - 63 tRNA
    - grep "   CDS     " -v annotation_CDS_RNA_hmms.gff | cut -f 3 | sort | uniq -c
      - yield the same information as the above script except without the count for CDS 
    - grep "kegg_ko" -c annotation_CDS_RNA_hmms.gff
      - how many sequences are annotated with a KEGG KO? 4031 
    - grep "kegg_ko" annotation_CDS_RNA_hmms.gff | sed 's#.*kegg_ko=\([^;]*\).*#\1#' | tr ',' '\n' | sort | uniq -c | sort -n
      - which KO is most common? 
    - Try to adapt the code above to the Pfam domains:
      - grep "pfam" annotation_CDS_RNA_hmms.gff | sed 's#.*pfam=\([^;]*\).*#\1#' | tr ',' '\n' | sort | uniq -c | sort -n
    - Try to find the cog annotations of the genes are annotated with the most common KO
      - grep "kegg_ko" annotation_CDS_RNA_hmms.gff | sed 's#.*cog=\([^;]*\).*#\1#' | tr ',' '\n' | sort | uniq -c | sort -n 
    - Try to find the Pfam annotations of the genes are annotated with the most common KO. Check the Pfam DB and KEGG DB for what these annotations mean. Do they correspond to a similar function?
      - grep "kegg_ko" annotation_CDS_RNA_hmms.gff | sed 's#.*pfam=\([^;]*\).*#\1#' | tr ',' '\n' | sort | uniq -c | sort -n
      - search the databases manually? 

## Lecture 8

- Metagenomics 101: Lecture 8: Functional Profiles
    - approaches to make (semi-)quantitative profiles of the molecular functions found in metagenomes, and how they can be compared across sambles
    - look back at mapping and annotation
    - Gene abundance measures - functional profile calculation
        - reads per gene/reads per functin
        - reads per kilobase
        - copies per million
        - average depth of coverage
    - working with functional profiles 
- Tutorial: counting reads and calculating profiles
    - see how IMP3 calculates functional profiles and the depth of coverage of genes 
    - files used: annotation_CDS_RNA_hmms.gff  
        - from annotation on Analysis/annotation directory
        - from Assembly directory: mg.reads.sorted.bam, mg.assembly.merged.fa, mg.reads.sorted.bam.bai
        - FeatureCounts: to extract the number of reads per called gene.
            - you supply the GFF file, which contains the positions of the genes (or open reading frames), and the .bam file, which contains for all the reads the positions where they map. Feature counts just counts how many reads map on each of the open reading frames.
            - output in mg.CDS_counts.tsv contains a line with the cull call to featureCounts and a header line and then one line for all features in the GFF file. As you can see in the header line, the first field in every line gives the actual feature and the last column contains the number of reads per feature.
        - Running featureCounts for KO profiles 
            - featureCounts can also get numbers of reads per level of the same annotations, e.g. KOs. To run the calculation for the open reading frames that were annotated with a KO, we first need to filter the GFF file, because featureCounts is a bit unflexible with its input:
                - grep "kegg_ko=" /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/PROFILES/annotation_CDS_RNA_hmms.gff >> annotation_KEGG.gff
            - Files with counts generated in this way can be used for differential abundance analysis, e.g. by DESeq2. Here is an example script to summarize featureCounts outputs from several assembled samples.
                - DESeq2: https://bioconductor.org/packages/devel/bioc/manuals/DESeq2/man/DESeq2.pdf 
                - script: https://github.com/a-h-b/mump/blob/main/workflow/scripts/multi_collect_featureCounts.R
        - Calculating average depth of coverage values
            - When working with functional profiles, sometimes you will be interested in how many copies you had of a gene and how abundant they are. For example, you may have identified a function of interest. With your GFF file, it is quite easy to identify the genes that have this function (see session 07). Now, you could look at the number of reads that map to each of those genes in the featureCounts output from the start of this lesson. But this only tells you how many reads overlapped. If the genes have different lengths, this is not the best measure of their abundance. In this case, you would want to have the average coverage:
            - use the same conda environment as before (used for featureCounts): conda activate /zfs/omics/projects/metatools/TOOLS/IMP3/conda/8fab667c26067cc13be382368594bb18
            - First, we use a small script to get the length of all contigs. We need this to make a nicely formatted file with all the genes.:
                - export PERL5LIB=$CONDA_PREFIX/lib/site_perl/5.26.2
                - perl /zfs/omics/projects/metatools/TOOLS/IMP3/workflow/scripts/fastaNamesSizes.pl  /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/PROFILES/mg.assembly.merged.fa > mg.assembly.length.txt
                - conda deactivate
            - For the next step, you need to use a different conda environment, which includes the bedtools suite. bedtools is good juggling coordinates (positions) on the assemblies.
                - bedtools: https://bedtools.readthedocs.io/en/latest/
                - conda activate /zfs/omics/projects/metatools/TOOLS/IMP3/conda/76e02d85877600c41ac84cb7bc27a189
                - This step will make a histogram file which contains for every gene the number of positions with a given coverage.
                    - sortBed -g mg.assembly.length.txt -i /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/PROFILES/annotation_CDS_RNA_hmms.gff |\
                        awk '$4 < $5' |\
                        coverageBed -hist -sorted -g mg.assembly.length.txt -b /zfs/omics/projects/metatools/SANDBOX/Metagenomics101/EXAMPLE_DATA/PROFILES/mg.reads.sorted.bam -a "stdin" |\
                        grep -v "^all" > helper_bedfile
                    - paste <(cat helper_bedfile | cut -f9 | cut -f1 -d ";" | sed -e "s/ID=//g") \
                        <(cut -f10,11,12,13 helper_bedfile) > mg.gene_depth.hist
                - A small script combines the histograms to an average value per gene:
                    - perl /zfs/omics/projects/metatools/TOOLS/IMP3/workflow/scripts/calcAvgCoverage.pl mg.gene_depth.hist > mg.gene_depth.avg
            - In addition to looking at the coverage for specific genes of interest, you can use this file for example for the kind of overvire visualization the IMP3 report gave you.